# 도구 호출 에이전트(Tool Calling Agent)

모델이 하나 이상의 도구(tool) 가 호출되어야 하는 시기를 감지할 수 있다.

쿼리가 해결될 때까지 반복적으로 도구를 호출하고 결과를 수신하는 에이전트를 만들 수 있다.

In [ ]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

Tool 세팅

In [2]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL


# 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


# 도구 생성
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result


print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

tools = [search_news, python_repl_tool]


도구 이름: search_news
도구 설명: Search Google News by input keyword
도구 이름: python_repl_tool
도구 설명: Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.


Agent 생성 (create_tool_calling_agent)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

Agent 실행기 (AgentExecutor)

In [4]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    max_execution_time=10,
    handle_parsing_errors=True,
)

# AgentExecutor 실행
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE1CRl9iOVFRYmdaNm9hT3pKZmZOVkZvcVRsRG5sMWk4bjhsWTBseTdDQ0FWVWZ3bmw1anI0aEQ1U3lqLUhHUDhzYUhqLUsycXQ4QTFpYUljRTlMQdIBVEFVX3lxTE42b2NyYUNaRWdrQkxHX2dRal92U25COTRqZnNUb0IyUDMxM2FPWEZwQzRYOTU0STVnUHdWbWV6NnRWVnQ4NFNRd1plZlNiTnFpdUVMRQ?oc=5', 'content': "삼성전자, 사상 최대 R&D·시설 투자…'AI 시대' 주도한다 - 한국경제"}, {'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE81QVlXam1wclZOV3loQndWUjVjU3FKUE9zMzRjbkU5b3RMSjFBaGVfMHBvM28wRDFyd05yMVhQYTVnZy1nVkhGZXBqLTV2Zks5bnFQdG8wVEtBeEJoZllhVkQwQ2d1MENteEE?oc=5', 'content': '오픈AI, 투자 유치 이후 글로벌 확장...뉴욕·파리·싱가포르 등 5곳에 지사 개설 - AI타임스'}, {'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTE4yZEVnMVBTeGNZWEZzQmpzeHhsZ2o5UTVSR3g2OXhNUTZLendsT2R1Zlc1ejUzd3VLaWk1LXhnOElyY3lRSkRfYm5aUGIzXzRja0lSRE5QUFhzUmfSAV5BVV95cUxOSXJ5YU1xU1lrYmpJQWZVRXoycHhxaVp2eWVNb0Q5aW1kTWtTZzlBdXVsaXpiUm5OUXNpMFB1WXMyWWd6c28tVHFzUXgyMVNY